# Coffee Machine

This lab is a tutorial on belief propagation, one of the many approaches for solving graphical models. The actual goal is to build a model for identifying faults with coffee machines. You're given a graphical model of how a coffee machine works, that connects observations to possible failure modes. Belief propagation is then used to calculate the probability of each failure (marginal probability) given the available evidence (observations). This is broken up as:
1. Given the graphical model for the state of a coffee machine learn the distributions from data. This is basic (Bayesian) statistics.
2. Implement belief propagation, so you can evaluate the probability of each failure given the available evidence. This is broken down into a number of steps:
    1. Conversion from _Bayes net_ to _factor_graph_, as BP is simplest to implement using the later. A factor graph is described in the slides, but is a bipartite graph of factors and random variables.
    2. Belief propagation works by passing messages along the edges of a factor graph. Each message depends on other messages having already been sent, so the second step is to work out a valid order to send messages in.
    3. Now you need to pass the messages, by implementing two functions: One to pass a message from a RV (random variable) to a factors, and another to pass a message from a factor to a RV.
    4. Finally, the above two steps let you pass all of the messages and extract the final belief. There is an extra complication to handle observed random variables. This has been coded for you.
3. Identify the most probable failure for a set of broken coffee machines. No marks here, just lets you try the algorithm out!

## Coffee Machine Data

Alongside this notebook you can also download a zip file that contains the data as a `csv` file. Code provided below will load the data directly from the zip file, so you don't have to unzip it.

Each row of the file contains a fully observed coffee machine, with the state of every random variable. The random variables are all binary, with `False` represented by `0` and `True` represented by `1`. The variables are:

Failures _(you're trying to detect these)_:
* 0. `he` - No electricity
* 1. `fp` - Fried power supply unit
* 2. `fc` - Fried circuit board  
* 3. `wr` - Water reservoir empty
* 4. `gs` - Group head gasket forms seal  
* 5. `dp` - Dead pump  
* 6. `fh` - Fried heating element  


Mechanism _(these are unobservable)_:
* 7. `pw` - Power supply unit works  
* 8. `cb` - Circuit board works  
* 9. `gw` - Get water out of group head   

Diagnostic _(these are the tests a mechanic can run - observable)_:
* 10. `ls` - Room lights switch on
* 11. `vp` - A voltage is measured across power supply unit
* 12. `lo` - Power light switches on
* 13. `wv` - Water visible in reservoir
* 14. `hp` - Can hear pump
* 15. `me` - Makes espresso
* 16. `ta` - Makes a hot, tasty espresso

Above the number is the column number of the provided data (`dm`, an abbreviation of _data matrix_) and the two letter code is a suggested variable name.

## Coffee Machine Model

For if you are unfamiliar with an espresso coffee machine here is a brief description of how one works (you can ignore this):
> The user puts ground coffee into a portafilter (round container with a handle and two spouts at the bottom), tamps it (compacts the coffee down), and clamps the portafilter into the group head at the front of the machine. A gasket (rubber ring) forms a seal between the portafilter and group head. A button is pressed. Water is drawn from a reservoir by a pump into a boiler. In the boiler a heating element raises the waters temperature, before the pump pushes it through the group head and into the portafilter at high pressure. The water is forced through the coffee grinds and makes a tasty espresso.

The graphical model (Bayes net) showing how the variables are related is also provided, as `coffee machine.pdf`; here it is given as conditional probabilities:

Failures:
 * `P_he:` $P(\texttt{no electricity})$
 * `P_fp:` $P(\texttt{fried psu})$
 * `P_fc:` $P(\texttt{fried circuit board})$
 * `P_wr:` $P(\texttt{water reservoir empty})$
 * `P_gs:` $P(\texttt{group head gasket seal broken})$
 * `P_dp:` $P(\texttt{dead pump})$
 * `P_fh:` $P(\texttt{fried heating element})$

Mechanism:
 * `P_pw_he_fp:` $P(\texttt{psu works}\enspace|\enspace\texttt{no electricity},\enspace\texttt{fried psu})$
 * `P_cb_pw_fc:` $P(\texttt{circuit board works}\enspace|\enspace\texttt{psu works},\enspace\texttt{fried circuit board})$
 * `P_gw_cb_wr_dp:` $P(\texttt{get water}\enspace|\enspace\texttt{circuit board works},\enspace\texttt{water reservoir empty},\enspace\texttt{dead pump})$

Diagnostic:
 * `P_ls_he:` $P(\texttt{lights switch on}\enspace|\enspace\texttt{no electricity})$
 * `P_vp_pw:` $P(\texttt{voltage across psu}\enspace|\enspace\texttt{psu works})$
 * `P_lo_cb:` $P(\texttt{power light on}\enspace|\enspace\texttt{circuit board works})$
 * `P_wv_wr:` $P(\texttt{water visible}\enspace|\enspace\texttt{water reservoir empty})$
 * `P_hp_dp:` $P(\texttt{can hear pump}\enspace|\enspace\texttt{dead pump})$
 * `P_me_gw_gs:` $P(\texttt{makes espresso}\enspace|\enspace\texttt{get water},\enspace\texttt{group head gasket seal broken})$
 * `P_ta_me_fh:` $P(\texttt{tasty}\enspace|\enspace\texttt{makes espresso},\enspace\texttt{fried heating element})$

Note that while the model is close to what you may guess the probabilities are not absolute, to account for mistakes and unknown failures. For instance, the mechanic may make a mistake while brewing an espresso and erroneously conclude that the machine is broken when it is in fact awesome. The probabilities associated with each failure are not uniform. The data set is roughly $50:50$ between failed/working machines, which is hardly realistic for a real product, but makes this exercise simpler as it avoids the problem of extremely rare failure modes.

In [1]:
%matplotlib inline

import zipfile
import io
import csv

from collections import defaultdict
import itertools

import numpy
import matplotlib.pyplot as plt



# Helper used below for some pretty printing, to loop all conditional variable combinations...
# (feel free to ignore)
def loop_conditionals(count):
    if count==0:
        yield (slice(None),)
    
    else:
        for head in loop_conditionals(count - 1):
            yield head + (0,)
            yield head + (1,)

## Loading Data

The below loads the data; it also includes some helpful variables.

In [2]:
# A mapping from the suggested variable names to column indices in the provided file...
nti = dict() # 'name to index'

nti['he'] = 0
nti['fp'] = 1
nti['fc'] = 2
nti['wr'] = 3
nti['gs'] = 4
nti['dp'] = 5
nti['fh'] = 6

nti['pw'] = 7
nti['cb'] = 8
nti['gw'] = 9

nti['ls'] = 10
nti['vp'] = 11
nti['lo'] = 12
nti['wv'] = 13
nti['hp'] = 14
nti['me'] = 15
nti['ta'] = 16



# Opposite to the above - index to name...
itn = ['he', 'fp', 'fc', 'wr', 'gs', 'dp', 'fh',
       'pw', 'cb', 'gw',
       'ls', 'vp', 'lo', 'wv', 'hp', 'me', 'ta'] # 'index to name'



# For conveniance this code loads the data from the zip file,
# so you don't have to decompress it (takes a few seconds to run)...
with zipfile.ZipFile('coffee_machines.zip') as zf:
    with zf.open('coffee_machines.csv') as f:
        sf = io.TextIOWrapper(f)
        reader = csv.reader(sf)
        next(reader)
        dm = []
        for row in reader:
            dm.append([int(v) for v in row])
        dm = numpy.array(dm, dtype=numpy.int8)



# Basic information...
print('Data: {} exemplars, {} features'.format(dm.shape[0], dm.shape[1]))
print('     Broken machines  =', dm.shape[0] - dm[:,nti['ta']].sum())
print('     Working machines =', dm[:,nti['ta']].sum())

Data: 262144 exemplars, 17 features
     Broken machines  = 122603
     Working machines = 139541


## Distribution Storage

The below defines storage to represent the probability distributions that are needed to complete the graphical model. You will be filling them in question 1 (below). A further variable contains a computer readable representation of these variables, for convenience.

In [3]:
# A set of variables that will ultimately represent conditional probability distributions.
# The naming convention is that P_he contains P(he), that P_ta_me_hw contains P(ta|me,hw) etc.
# Indexing is always in the same order that the variables are given in the variables name.

P_he          = numpy.zeros(2)
P_fp          = numpy.zeros(2)
P_fc          = numpy.zeros(2)
P_wr          = numpy.zeros(2)
P_gs          = numpy.zeros(2)
P_dp          = numpy.zeros(2)
P_fh          = numpy.zeros(2)

P_pw_he_fp    = numpy.zeros((2,2,2))
P_cb_pw_fc    = numpy.zeros((2,2,2))
P_gw_cb_wr_dp = numpy.zeros((2,2,2,2))

P_ls_he       = numpy.zeros((2,2))
P_vp_pw       = numpy.zeros((2,2))
P_lo_cb       = numpy.zeros((2,2))
P_wv_wr       = numpy.zeros((2,2))
P_hp_dp       = numpy.zeros((2,2))
P_me_gw_gs    = numpy.zeros((2,2,2))
P_ta_me_fh    = numpy.zeros((2,2,2))



# This list describes the above in a computer readable form,
# as the tuple (numpy array, human readable name, list of RVs, kind);
# the list of RVs is aligned with the dimensions of the numpy array
# and kind is 'F' for failure, 'M' for mechanism and 'D' for diagnostic..
rvs = [(P_he, 'P(he)', [nti['he']], 'F'),
       (P_fp, 'P(fp)', [nti['fp']], 'F'),
       (P_fc, 'P(fc)', [nti['fc']], 'F'),
       (P_wr, 'P(wr)', [nti['wr']], 'F'),
       (P_gs, 'P(gs)', [nti['gs']], 'F'),
       (P_dp, 'P(dp)', [nti['dp']], 'F'),
       (P_fh, 'P(fh)', [nti['fh']], 'F'),
       (P_pw_he_fp, 'P(pw|he,fp)', [nti['pw'], nti['he'], nti['fp']], 'M'),
       (P_cb_pw_fc, 'P(cb|pw,fc)', [nti['cb'], nti['pw'], nti['fc']], 'M'),
       (P_gw_cb_wr_dp, 'P(gw|cb,wr,dp)', [nti['gw'], nti['cb'], nti['wr'], nti['dp']], 'M'),
       (P_ls_he, 'P(ls|he)', [nti['ls'], nti['he']], 'D'),
       (P_vp_pw, 'P(vp|pw)', [nti['vp'], nti['pw']], 'D'),
       (P_lo_cb, 'P(lo|cb)', [nti['lo'], nti['cb']], 'D'),
       (P_wv_wr, 'P(wv|wr)', [nti['wv'], nti['wr']], 'D'),
       (P_hp_dp, 'P(hp|dp)', [nti['hp'], nti['dp']], 'D'),
       (P_me_gw_gs, 'P(me|gw,gs)', [nti['me'], nti['gw'], nti['gs']], 'D'),
       (P_ta_me_fh, 'P(ta|me,fh)', [nti['ta'], nti['me'], nti['fh']], 'D')]


## 1. Learning Conditional Probability Distributions

Above a set of variables representing conditional probability distributions has been defined. They are to represent a Bernoulli trial for each combination of conditional variables, given as $P(\texttt{False}|...)$ in `rv[0,...]` and $P(\texttt{True}|...)$ in `rv[1,...]`. Obviously these two values should sum to $1$; giving the probability of both `True` and `False` is redundant, but makes all of the code much cleaner.

Your task is to fill in the distributions with a maximum a posteriori probability (MAP) estimate given the data. The prior to be used for all RVs is a Beta distribution,

$$P(x) \propto x^{\alpha-1}(1-x)^{\beta-1}$$

such that $x$ is the probability of getting a `False` from the Bernoulli draw (note that this is backwards from what you might suspect, because it keeps the arrays in the same order). The hyperparameters for the priors are to be set as $\alpha = \beta = 1$. The Beta distribution is _conjugate_, that is when you observe a Bernoulli draw and update using Bayes rule the posterior is also a Beta distribution. This makes the procedure particularly simple:
1. Initialise every conditional probability with the hyperparameters $\alpha$ and $\beta$
2. Update them for every coffee machine in the data set
3. Extract the maximum likelihood parameters from the posterior $\alpha$ and $\beta$ parameters

Writing out the relevant parts of the Bayes rule update for observing a RV, $v = 0$ (`False`), you get

$$\operatorname{Beta}(x | \alpha_1, \beta_1) \propto \operatorname{Bernoulli}(v = 0 | x)\operatorname{Beta}(x | s\alpha_0,\beta_0)$$

$$x^{\alpha_1-1}(1-x)^{\beta_1-1} \propto \left(x^{(1-v)} (1-x)^v\right) \left(x^{\alpha_0-1}(1-x)^{\beta_0-1}\right)$$

$$x^{\alpha_1-1}(1-x)^{\beta_1-1} \propto x^1 (1-x)^0 x^{\alpha_0-1}(1-x)^{\beta_0-1}$$

$$x^{\alpha_1-1}(1-x)^{\beta_1-1} \propto x^{\alpha_0+1-1}(1-x)^{\beta_0-1}$$

$$\operatorname{Beta}(x | \alpha_1, \beta_1) = \operatorname{Beta}(x | \alpha_0+1,\beta_0)$$

Subscripts of the hyperparameters are used to indicate how many data points have been seen; `True` works similarly. Put simply, the result is that you count how many instances exist of each combination, and add $1$ for the hyperparameters. The maximum likelihood is then the expected value, which is

$$P(v=\textrm{False}|\ldots) = \frac{\alpha}{\alpha + \beta}$$

It's typical to do all of the above within the conditional probability distribution arrays, using them to represent $\alpha$ and $\beta$ then treating step 3 as converting from hyperparameters to expected values.

Hints:
* The use of `0=False` and `1=True` both in the `dm` array and in the conditional probability distributions is very deliberate.
* Do not write unique code for each variable - that will be hundreds of lines of code and very tedious/error prone. It's possible to get all of the marks in 7 lines of code, or less, if you're particularly sneaky.
* Remember that you can index a `numpy` array with a tuple; for instance using a tuple comprehension such as `tuple(dm[row,c] for c in columns)` you could index an array with the values of the given `columns` indices, as extracted from the current `row` of the data matrix.
* The provided `rvs` array exists to support the above two hints.
 
__(2 marks)__

In [4]:
# **************************************************************** 2 marks
for rv in rvs:
    rv[0].fill(1)

for i in range(len(dm)):
    for rv in rvs:
        rv[0][tuple(dm[i, c] for c in rv[2])] += 1
    
for rv in rvs:
    p = rv[0]
    p /= numpy.sum(p, axis=0)


# Print out the RVs for a sanity check...
for rv in rvs:
    if len(rv[2])==1:
        print('{} = {}'.format(rv[1], rv[0]))
    else:
        print('{}:'.format(rv[1]))
        for i in loop_conditionals(len(rv[2])-1):
            print('  P({}|{}) = {}'.format(itn[rv[2][0]], ','.join([str(v) for v in i[1:]]), rv[0][i]))
    print()

P(he) = [0.99047096 0.00952904]

P(fp) = [0.80056152 0.19943848]

P(fc) = [0.98004547 0.01995453]

P(wr) = [0.89956742 0.10043258]

P(gs) = [0.95010414 0.04989586]

P(dp) = [0.94968071 0.05031929]

P(fh) = [0.97033714 0.02966286]

P(pw|he,fp):
  P(pw|0,0) = [4.81037502e-06 9.99995190e-01]
  P(pw|0,1) = [9.99980683e-01 1.93173257e-05]
  P(pw|1,0) = [9.99495714e-01 5.04286435e-04]
  P(pw|1,1) = [0.9980695 0.0019305]

P(cb|pw,fc):
  P(cb|0,0) = [9.99981208e-01 1.87916941e-05]
  P(cb|0,1) = [9.99048525e-01 9.51474786e-04]
  P(cb|1,0) = [4.90910787e-06 9.99995091e-01]
  P(cb|1,1) = [9.99760937e-01 2.39062874e-04]

P(gw|cb,wr,dp):
  P(gw|0,0,0) = [9.99980048e-01 1.99517168e-05]
  P(gw|0,0,1) = [9.99611349e-01 3.88651380e-04]
  P(gw|0,1,0) = [9.99817718e-01 1.82282173e-04]
  P(gw|0,1,1) = [0.99630996 0.00369004]
  P(gw|1,0,0) = [5.75251529e-06 9.99994247e-01]
  P(gw|1,0,1) = [9.99892404e-01 1.07596299e-04]
  P(gw|1,1,0) = [0.90239779 0.09760221]
  P(gw|1,1,1) = [9.99056604e-01 9.43396226e-04]

## 2. Factor Graph

The graphical model has been given as a _Bayesian network_, but computation is much simpler on a _factor graph_ (see slides for details, including a visual version of the below algorithm). Inevitably, the first step is to convert; the algorithm to do so is:

1. For each RV we need two nodes: The random variable itself and a factor node. There must be an edge connecting them.
2. Each conditional distribution generates as many edges as there are conditional terms. Each edge is between the factor associated with the RV and one of the RVs it is conditioned on.

The algorithm itself involves passing messages along the edges, so what we require is a list of edges. Being more specific, the objective is hence to generate that list of edges; an edge is represented by a tuple containing two integers, these being the indices of the nodes that are either side of the edge. For the RVs we can use the indices already provided by `nti` and used in the `rvs` list. As factors are paired with RVs the index of the factor node paired with RV $i$ shall be defined $i+17$ (there are $17$ RVs, so this puts them immediately after the RVs in the same order).

__(1 mark)__

In [5]:
def rv2fac(i):
    """Given the index of a random variable node this returns the index of
    the corresponding factor node."""
    return i + 17


def fac2rv(i):
    """Given the index of a factor node this returns the index of
    the corresponding random variable node."""
    return i - 17



def calc_edges(rvs):
    """Uses the rvs array to calculate and return a list of edges,
    each as a tuple (index node a, index node b)"""
    edges = []
    # **************************************************************** 1 mark
    all_rvs = [i[2] for i in rvs]
    for arv in all_rvs:
        if len(arv) == 2:
            edges.append((arv[1], arv[0]))
        else:
            temp = arv[0]
            for i in arv[1:]:
                edges.append((i, temp))
    
    return edges


edges = calc_edges(rvs)
print('Generated {} edges'.format(len(edges)))

Generated 16 edges


## 3. Message Order

Belief propagation works by passing messages over the edges of the factor graph, two messages per edge as there is one in each direction. A message can only be sent from node `a` to node `b` when node `a` has received all of it's messages, __except__ for the message from node `b` to node `a`, which is not needed for that calculation. For this reason we need to convert the list of edges into a list of messages to send, ordered such that the previous constraint is never violated. Note that this naturally involves doubling the number of items in the list as we now have two entries per edge, one per direction.

There are many ways to solve this problem and you're welcome to choose any (the below is not the best), but here is a description of an approach that works by simulating message passing where constraints are checked: Generate a list of messages to send (the edges plus the edges with the tuples reversed, to cover both directions) and an empty list, which will ultimately contain the messages in the order they were sent. The algorithm then proceeds by identifying messages in the first list for which the constraint has been satisfied and then moving them to the end of the second list, indicating the message has been sent. This is repeated until the first list is empty and the second full; the second will be in a valid message sending order.

Implemented badly the above is both horrifically slow and fiddly to code. One technique to make it faster is to _bounce_ — instead of going through the list of messages to send in the same order each time you go forwards and then backwards and then forwards etc. This avoids the scenario where the to send list contains the messages in reverse sending order, such that with each loop you only find one more message that you can send, meaning you have to loop as many times as there are messages. A second technique also makes the code much simpler. Figuring out if you can send a message when the lists are represented as lists is slow — you have to loop both. Instead, convert the lists into a pair of nested dictionaries indexed `[message destination][message source]` (order is important!) that contains `True` if the message has been sent (in second list), `False` if it has not (in first list). It's now simple to check if a message can be sent or not; you will still need to generate the list of messages to send as you flip values from `False` to `True` (make sure you get the order right — the dictionaries are indexed backwards).

Hint:
* When converting from edges to dictionaries of flags `defaultdict(dict)` may simplify.
* `all(v for v in thing if <condition>)` is valid Python, that returns `True` only if `v` is `True` for every instance that passes the condition.

__(3 marks)__

In [15]:
def calc_msg_order(edges):
    """Given a list of edges converts to a list of messages such that
    the returned list contains tuples of (source node, destination node)."""
    msgs = []
    # **************************************************************** 3 marks
    new_list = edges + [i[::-1] for i in edges]
    
    for i in new_list:
        print(i)
        
    return msgs



msg_order = calc_msg_order(edges)
print('Generated {} messages'.format(len(msg_order)))

(0, 7)
(1, 7)
(7, 8)
(2, 8)
(8, 9)
(3, 9)
(5, 9)
(0, 10)
(7, 11)
(8, 12)
(3, 13)
(5, 14)
(9, 15)
(4, 15)
(15, 16)
(6, 16)
(7, 0)
(7, 1)
(8, 7)
(8, 2)
(9, 8)
(9, 3)
(9, 5)
(10, 0)
(11, 7)
(12, 8)
(13, 3)
(14, 5)
(15, 9)
(15, 4)
(16, 15)
(16, 6)
Generated 0 messages


## 4. Message Passing

There are two kinds of node: RVs and factors. The message sent by a random variable is simply all incoming messages multiplied together, except for the message from the direction it is sending a message, i.e.

$$M_{s \rightarrow d}(x) = \prod_{\forall n \cdot n \neq d} M_{n \rightarrow s}(x)$$

where $s$ (source) is a RV and $d$ (destination) is a factor and $n$ covers the neighbours of $s$ (they must each have sent a message to $s$, and will be factors). $x$ is $0$ or $1$, corresponding to `False` or `True`; messages are functions, conveniently discrete and hence represented as arrays in this case. This equation is exactly what you need to evaluate in `send_rv()`. An almost identical equation is the _belief_, that is the marginal distribution of a RV and the final output of the algorithm:

$$B_s(x) = \prod_{\forall n} M_{n \rightarrow s}(x)$$

In the interest of laziness `send_rv` should calculate this if the destination is set to `None`; if implemented in the simplest way this will naturally be the case.


The messages that factors send are more complicated, because they _factor_ in (sorry) the conditional probability distributions:

$$M_{s \rightarrow d}(x_d) = \sum_{\forall x_m \cdot m \neq d} P[s,d,\ldots] \prod_{\forall n \cdot n \neq d} M_{n \rightarrow s}(x_n)$$

where $P[s,d,\ldots]$ is the conditional probability distribution, which will naturally be indexed by the source, destination, and any other neighbours ($n$). The switch to $[]$ is to indicate that you should stop thinking of it as a probability distribution when passing messages, as some of the messages make no sense when interpreted as such (but the final beliefs always make sense; it's just the intermediate messages that get weird). The key detail is that this is no longer simple multiplication: Each message is over a different RV (the RV of its source) and hence needs to be multiplied in the correct way. A typical recipe for this is:
1. Copy the factor ($P[s,d,\cdot]$) so it can be used as working storage
2. Multiply in each message to the source (unless from destination), using broadcasting to align it with the correct dimension
3. Marginalise out all but the RV of the destination node

Hints:
* Messages are always length 2 vectors, at least in this scenario where everything is a binary RV.
* Remember that the factor index in `send_factor` is offset by $17$ from the index of the actual factor it needs to use in `rvs`.
* [`einsum()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.einsum.html) makes `send_factor` very elegant, if you dare (can all be done in one, horrifying, line).
* Alternatively, [`reshape()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html) is easier to understand for multiplication and then [`sum()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.sum.html) lets you sum out (marginalise) many axes at the same time, by giving them as a tuple to the `axes` keyword parameter.

__(4 marks)__

In [ ]:
def send_rv(src, dest, msgs):
    """Returns the message to send from src (source, always a RV) to dest (destination,
    always a factor). msgs is dictionaries within a dictionary such that [d][s]
    gets you the message from s to d. Because the message sending order is
    correct (well...) you can assume all required entries exist in msgs."""

    # **************************************************************** 1 mark
    
    pass



def send_factor(src, dest, msgs, rvs):
    """Returns the message to send from src (source, always a factor) to dest
    (destination, always a RV). msgs is dictionaries within a dictionary such
    that [d][s] gets you the message from s to d. Because the message sending
    order is correct you can assume all required entries exist in msgs.
    rvs is as defined above and contains the relevant conditional distributions
    and the names of the dimensions"""
    
    # **************************************************************** 3 marks
    
    pass


## Belief Propagation

Once you have the ability to send messages and an order in which to send the messages the rest of the algorithm is a cake walk. The only trick is that if a RV is known (observed) then instead of using `send_rv` whenever a message is sent from it you send the distribution it is known to be instead (`[1,0]` for `False`, `[0,1]` for `true`). The below code implements this.

In [ ]:
def marginals(known):
    """known is a dictionary, where a random variable index existing as a key in the dictionary
    indicates it has been observed. The value obtained using the key is the value the
    random variable has been observed as. Returns a 17x2 matrix, such that [rv, 0] is the
    probability of random variable rv being False, [rv, 1] the probability of being True."""
    
    # Message storage...
    msgs = defaultdict(dict) # [destination][source] -> message
    
    # Message passing...
    for src, dest in msg_order:
        if src < 17:
            # Random variable...
            if src not in known:
                msgs[dest][src] = send_rv(src, dest, msgs)
            
            else:
                msgs[dest][src] = numpy.array([0,1] if known[src] else [1,0])
        
        else:
            # Factor...
            msgs[dest][src] = send_factor(src, dest, msgs, rvs)
    
    # Calculate and return beliefs/marginal distributions...
    ret = numpy.empty((17,2))
    for r in range(ret.shape[0]):
        if r not in known:
            ret[r,:] = send_rv(r, None, msgs)
            ret[r,:] /= ret[r,:].sum() # This needed due to fix numerical stability issues
        
        else:
            ret[r,:] = numpy.array([0,1] if known[r] else [1,0])

    return ret



print('Marginals with no observations:') # P(ta = True) = 0.53288705
belief = marginals({})
for i in range(belief.shape[0]):
    print('  P({}) = {}'.format(itn[i], belief[i,:]))
print()


print('Marginals if 𝚠𝚊𝚝𝚎𝚛 𝚛𝚎𝚜𝚎𝚛𝚟𝚘𝚒𝚛 𝚎𝚖𝚙𝚝𝚢:') # P(ta = True) = 0.05732075
belief = marginals({nti['wr'] : True})
for i in range(belief.shape[0]):
    print('  P({}) = {}'.format(itn[i], belief[i,:]))
print()

## What's broken?

The below is not worth any marks, but it would be weird to implement an algorithm and then never run it! Simply print out what the most probable broken part of each of the below machines is.

In [ ]:
repair = {}
repair['A'] = {nti['me'] : True}
repair['B'] = {nti['wv'] : True}
repair['C'] = {nti['wv'] : False, nti['lo'] : True}
repair['D'] = {nti['hp'] : False, nti['ta'] : False}
repair['E'] = {nti['hp'] : True, nti['wv'] : True, nti['vp']: True}

# **************************************************************** 0 marks
